In [1]:
import math
import tensorflow as tf
import numpy as np
import pylab as plt

/Users/lzcai/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# scale data
def scale(X, X_min, X_max):
    return (X - X_min)/(X_max-X_min)

In [3]:
def process_inputs_from_file(fileName): # Read in data
    inputs = np.loadtxt(fileName, delimiter=' ')
    X, _Y = inputs[:, :NUM_FEATURES], inputs[:, -1].astype(int)
    X = scale(X, np.min(X, axis=0), np.max(X, axis=0))
    _Y[_Y == 7] = 6 # Actually dont have, just in case have error data

    Y = np.zeros((_Y.shape[0], NUM_CLASSES))
    Y[np.arange(_Y.shape[0]), _Y - 1] = 1 #one hot matrix
    return X, Y

In [17]:
def init_weights(feature_no, neuron_no, name, logistic = True):
    # From eg.5.2
    n_in = feature_no
    n_out = neuron_no
    W_values = np.asarray(np.random.uniform(low=-np.sqrt(6. / (n_in + n_out)),
                                            high=np.sqrt(6. / (n_in + n_out)),
                                            size=(n_in, n_out)))
    if logistic == True:
        W_values *= 4
    return(tf.Variable(W_values, dtype=tf.float32, name=name))

def init_bias(neuron_no, name):
    # From eg.5.2
    return(tf.Variable(np.zeros(neuron_no), dtype=tf.float32, name=name))

In [149]:
# Initialize Values
NUM_FEATURES = 36
NUM_CLASSES = 6
NUM_HIDDEN = 10

LEARNING_RATE = 0.01
EPOCHS = 5000
BATCH_SIZE = 32
NUM_NEURONS = 10
SEED = 10
BETA = pow(10, -6)
np.random.seed(SEED)

TRAIN_FILE_NAME = 'sat_train.txt'
TEST_FILE_NAME = 'sat_test.txt'

In [6]:
trainX, trainY = process_inputs_from_file(TRAIN_FILE_NAME)
testX, testY = process_inputs_from_file(TEST_FILE_NAME)

In [21]:
def setup_cross_entropy(labels, logits):
    return tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits)

def setup_correct_prediction(labels, logits):
    return tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1)), tf.float32)

In [90]:
x = tf.placeholder(tf.float32, [None, NUM_FEATURES], name='x')
d = tf.placeholder(tf.float32, [None, NUM_CLASSES], name='d')

In [91]:
with tf.variable_scope("Hidden_layer"):
    W = init_weights(NUM_FEATURES, NUM_HIDDEN, name="Weight_1")
    b = init_bias(NUM_HIDDEN, name="Bias_1")
    z = tf.matmul(x, W) + b #syn_input_1
    h = tf.nn.sigmoid(z) #out_1

with tf.variable_scope("Output_layer"):
    V = init_weights(NUM_HIDDEN, NUM_CLASSES, name="Weight_2")
    c = init_bias(NUM_CLASSES, name="Bias_2" )
    y = tf.matmul(h, V) + c #syn_out_2
    f = y #out_2

cross_entropy = setup_cross_entropy(labels=d, logits=y)
regularization = tf.nn.l2_loss(V) + tf.nn.l2_loss(W) 
J = tf.reduce_mean(cross_entropy + BETA * regularization)

optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
train_op = optimizer.minimize(J)

correct_prediction = setup_correct_prediction(labels=d, logits=y)
accuracy = tf.reduce_mean(correct_prediction)

In [92]:
print(V)

<tf.Variable 'Output_layer_5/Weight_2:0' shape=(10, 6) dtype=float32_ref>


In [146]:
def generate_batch(X_in, y_in, batch_size):
    X = list(X_in)
    y = list(y_in)
    
    if len(X)!=len(y):
        print("Error: len(X)!=len(Y)")
        return None
    batched_X = []
    batched_y = []
    count = 0
    while (len(X) >= batch_size):
        batched_X.append(X[0:batch_size])
        del X[0:batch_size]
        batched_y.append(y[0:batch_size])
        del y[0:batch_size]
        if count % 10 == 0:
            print (count)
        count += 1
    
    if len(X) != 0:
        remain = batch_size-len(X)
        X.extend(batched_X[0][0:remain])
        y.extend(batched_y[0][0:remain])
        batched_X.append(X)
        batched_y.append(y)
        print("Remain rescaled to", len(X))
    
    return (batched_X, batched_y)
        

In [147]:
X, y = generate_batch(trainX, trainY, BATCH_SIZE)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
Remain rescaled to 32


In [158]:
len(X)

139

In [155]:
y[0]

[array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 0., 1., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0.]),
 array([0., 0.

In [156]:
trainX[0]

array([0.8125    , 0.8       , 0.76190476, 0.50413223, 0.70967742,
       0.68181818, 0.58947368, 0.390625  , 0.6875    , 0.72815534,
       0.54736842, 0.421875  , 0.98387097, 0.95192308, 0.86956522,
       0.59322034, 0.8125    , 0.82524272, 0.74698795, 0.41463415,
       0.69230769, 0.73786408, 0.6       , 0.416     , 0.96875   ,
       0.95192308, 0.93103448, 0.58333333, 0.75384615, 0.93069307,
       0.82105263, 0.5546875 , 0.6875    , 0.77669903, 0.66315789,
       0.453125  ])

In [157]:
trainY[0]

array([0., 0., 1., 0., 0., 0.])

In [153]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
train_acc = []

print("********* BEFORE **********")
print(W.eval())
print(b.eval())
print(V.eval())
print(c.eval())
print("***************************")
for i in range(EPOCHS):
#     rand_index = np.random.choice(int(len(trainX)/BATCH_SIZE)*2, size=BATCH_SIZE)
#     x_batch = trainX[rand_index]  # Transpose to the correct shape
#     y_batch = trainY[rand_index]
    for j in range(len(X)): # X[j]: One batch with 32 records
        train_op.run(feed_dict={x: X[j], d: y[j]})
    train_acc.append(accuracy.eval(feed_dict={x: X[j], d: y[j]}))

    if i % 10 == 0:
        print('iter %d: accuracy %g'%(i, train_acc[i]))

print("********* AFTER **********")
print(W.eval())
print(b.eval())
print(V.eval())
print(c.eval())
print("**************************")

/Users/lzcai/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


********* BEFORE **********
[[-0.1631836   0.60266995  0.05311186 -1.2281368   0.7421102   0.47391963
  -1.1101195   0.3278026   0.5335475  -1.0941672 ]
 [-0.95174646 -1.4321867  -0.6175264  -1.341087   -1.4412092  -0.19351709
   1.2099687   1.358986    1.0395935  -0.172818  ]
 [ 0.7887703   0.00421227  1.2860628   0.40760723  0.32334036 -0.25150952
  -0.09254244 -1.2869086   0.5898286  -0.8280431 ]
 [-0.6399256  -0.67013997  0.086604   -0.7004537  -0.22075005 -0.4225171
   0.06603097  0.11168236  1.1918697  -1.340074  ]
 [-0.5874912  -1.0822691  -1.0236305   0.96170086  0.15750325 -0.5641514
  -0.29894337 -0.17243817  0.10877943 -0.53368974]
 [ 0.40519494  0.13645941 -1.1735432  -1.3089533   0.90199685 -0.9147438
   1.0966138  -0.06500351  0.11330171  0.7744566 ]
 [-0.8272105   1.2012744  -0.5976823  -0.21788606  0.12646599 -0.54623485
   1.0002564  -0.4108801  -0.55163205  0.2952577 ]
 [ 1.1802617   1.2018846   0.7811596   0.25231975  0.5316893   0.84000325
  -0.7029413  -0.47306284 

KeyboardInterrupt: 

In [98]:
print("********* BEFORE **********")
print(W.eval())
print(b.eval())
print(V.eval())
print(c.eval())
print("***************************")
output_2_, accuracy_ = sess.run([f, accuracy], feed_dict={x: testX, d: testY})
print(output_2_, '\n',accuracy_)

********* BEFORE **********
[[-0.19989009  0.4903861   0.234219   -0.9959286   0.5981305   0.59950495
  -1.1553173   0.4607051   0.5899705  -1.2237152 ]
 [-1.0255724  -1.5502046  -0.8979562  -1.1498064  -1.5834476  -0.05036698
   1.2605588   1.3967464   1.2451229  -0.2405093 ]
 [ 0.7250079  -0.03535295  1.6924105   0.37279528  0.27839866 -0.14443174
  -0.07551167 -0.87984544  0.34147844 -0.9406678 ]
 [-0.65998584 -0.7080038   0.35340714 -0.71683925 -0.35197163 -0.34741756
   0.16737023  0.0543531   1.1219585  -1.2312447 ]
 [-0.6387123  -1.1896455  -0.82239586  1.1639997   0.06220909 -0.435247
  -0.32398522  0.11517991  0.07830356 -0.744543  ]
 [ 0.3177965   0.02433485 -1.4240621  -1.149511    0.8067215  -0.7740776
   1.1521646   0.12553705  0.23013231  0.63372016]
 [-0.9051388   1.1595538  -0.44105634 -0.27290526  0.06549148 -0.4334708
   1.0947609  -0.12521793 -0.71145755  0.25680473]
 [ 1.153712    1.1645901   1.015659    0.227401    0.39099166  0.9173834
  -0.6169094  -0.537867    1

In [14]:
trainX[0]

array([0.8125    , 0.8       , 0.76190476, 0.50413223, 0.70967742,
       0.68181818, 0.58947368, 0.390625  , 0.6875    , 0.72815534,
       0.54736842, 0.421875  , 0.98387097, 0.95192308, 0.86956522,
       0.59322034, 0.8125    , 0.82524272, 0.74698795, 0.41463415,
       0.69230769, 0.73786408, 0.6       , 0.416     , 0.96875   ,
       0.95192308, 0.93103448, 0.58333333, 0.75384615, 0.93069307,
       0.82105263, 0.5546875 , 0.6875    , 0.77669903, 0.66315789,
       0.453125  ])

In [16]:
x = tf.placeholder(tf.float32, shape=[None, NUM_FEATURES])
linear_model = tf.layers.Dense(units=1)
first_layer = linear_model(x)